# Year 2023 Day 11


In [157]:
import numpy as np
import pandas as pd
from pyobsplot import Plot # , js # can be used to pass js expr as str 
import xarray as xr


Note: the following logic can be reused for 202324 (show the vectors aligning progressively)

In [158]:
from advent_of_code.y_2023.problem_202311 import AdventOfCodeProblem202311

problem = AdventOfCodeProblem202311()
problem

AdventOfCodeProblem202311(year=2023, day=11)

In [159]:
puzzle_input = problem.parse_input_text_file()
puzzle_input

<xarray.DataArray (row: 140, col: 140)>
array([[46, 46, 46, ..., 46, 46, 46],
       [46, 46, 46, ..., 46, 46, 46],
       [46, 46, 46, ..., 46, 46, 46],
       ...,
       [46, 46, 46, ..., 46, 46, 46],
       [46, 46, 46, ..., 46, 46, 46],
       [46, 46, 46, ..., 46, 35, 46]], dtype=uint8)
Coordinates:
  * row      (row) int64 0 1 2 3 4 5 6 7 8 ... 132 133 134 135 136 137 138 139
  * col      (col) int64 0 1 2 3 4 5 6 7 8 ... 132 133 134 135 136 137 138 139

In [160]:
grid = (puzzle_input == ord("#")).astype(int) * 255
grid

<xarray.DataArray (row: 140, col: 140)>
array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0, 255,   0]])
Coordinates:
  * row      (row) int64 0 1 2 3 4 5 6 7 8 ... 132 133 134 135 136 137 138 139
  * col      (col) int64 0 1 2 3 4 5 6 7 8 ... 132 133 134 135 136 137 138 139

In [161]:
from advent_of_code.y_2023.problem_202311 import get_compartiments




In [162]:
from advent_of_code.y_2023.problem_202311 import (
    compute_adjacency_matrix,
    create_coord_array,
)


def visualize_puzzle_input(
    space_xda: xr.DataArray, with_rules: bool = False, with_graph: bool = False,
    coord_array:xr.DataArray|None = None,
):
    marks = []
    marks.append(
        Plot.raster(
            grid.values.reshape(-1),
            {
                "width": grid.col.size,
                "height": grid.row.size,
                "imageRendering": "pixelated",
            },
        ),
    )

    if with_rules:
        row_chunks = get_compartiments(space_xda, "row", "col")
        col_chunks = get_compartiments(space_xda, "col", "row")

        marks.append(
            Plot.ruleY(
                [chunk.start - 0.5 for chunk in list(row_chunks.values())[1:]],
                {"stroke": "red"},
            ),
        )
        marks.append(
            Plot.ruleX(
                [chunk.start - 0.5 for chunk in list(col_chunks.values())[1:]],
                {"stroke": "red"},
            ),
        )

    if with_graph:
        if coord_array is None:
            raise ValueError("coord_array was not given, it is needed to visualize the graph")
        
        coord_array = coord_array + 0.5

        lines = []
        for idx, pair_source in enumerate(coord_array.values.tolist()):
            for pair_target in coord_array.values[idx + 1 :].tolist():
                lines.append([pair_source, pair_target])

        # Only display a subset of all the links
        lines = lines[::500]
        
        table = {
            "x1": [],
            "y1": [],
            "x2": [],
            "y2": [],
        }

        for line in lines:
            table["y1"].append(line[0][0])
            table["x1"].append(line[0][1])
            table["y2"].append(line[1][0])
            table["x2"].append(line[1][1])

        table = pd.DataFrame.from_dict(table)
        marks.append(
            Plot.link(
                table,
                {
                    "x1": "x1",
                    "y1": "y1",
                    "x2": "x2",
                    "y2": "y2",
                    "stroke": "white",
                    "strokeWidth": 0.3,
                },
            )
        )

    return Plot.plot(
        {
            "height": 140 * 4,
            "width": 140 * 4,
            "color": {"scheme": "magma"},
            "x": {"domain": [0, grid.col.size], "label": "column"},
            "y": {"domain": [grid.row.size, 0], "label": "row"},
            "marks": marks,
        }
    )

In [163]:
visualize_puzzle_input(puzzle_input)

ObsplotWidget(spec={'data': [], 'code': {'height': 560, 'width': 560, 'color': {'scheme': 'magma'}, 'x': {'dom…

In [164]:
visualize_puzzle_input(puzzle_input, with_rules=True)

ObsplotWidget(spec={'data': [], 'code': {'height': 560, 'width': 560, 'color': {'scheme': 'magma'}, 'x': {'dom…

Let's count how many nodes there are in this graph:

In [165]:
coord_array = create_coord_array(puzzle_input)
node_count = coord_array["z"].size
node_count

425

Now we can calculate the number of edges in the related [Complete graph](https://en.wikipedia.org/wiki/Complete_graph), where every pair of node is connected:

In [166]:
n = node_count
edge_count = n*(n-1)//2 # 2 among n
edge_count

90100

The amount is very high, hence in the visualisation, only a fraction is shown

In [167]:
visualize_puzzle_input(
    puzzle_input, with_rules=True, with_graph=True, coord_array=coord_array
)

ObsplotWidget(spec={'data': [{'pyobsplot-type': 'DataFrame', 'value': b'ARROW1\x00\x00\xff\xff\xff\xff\x18\x04…

The key to expansion is to assign to every chunk a node (it creates a [Lattice graph](https://en.wikipedia.org/wiki/Lattice_graph)).
Then the same logic can be applied on chunks instead of stars, and the distance
becomes the sum of both adjacency matrix, with the chunk one multiplied by
the expansion coefficient